In [1]:
# !git clone https://github.com/naver/r2d2.git
# import os
# os.chdir('r2d2')

Cloning into 'r2d2'...
remote: Enumerating objects: 94, done.
remote: Total 94 (delta 0), reused 0 (delta 0), pack-reused 94
Unpacking objects: 100% (94/94), done.
     |████████████████████████████████| 112kB 18.2MB/s 


In [2]:
# %%writefile /content/Vis_Odometry/r2d2_utils.py
import sys
sys.path.insert(1, 'r2d2')
import os, pdb
from PIL import Image
import numpy as np
import torch
import glob
from tools import common
from tools.dataloader import norm_RGB
from nets.patchnet import *
import time
from skimage.util.shape import view_as_windows
from scipy.optimize import least_squares
import copy

import numpy as np
import cv2
import torch
import glob
import pickle
import matplotlib.pyplot as plt


def mnn_matcher(descriptors_a, descriptors_b, threshold = 0.9):
    device = descriptors_a.device
    sim = descriptors_a @ descriptors_b.t()
    nn_sim, nn12 = torch.max(sim, dim=1)
    nn21 = torch.max(sim, dim=0)[1]
    ids1 = torch.arange(0, sim.shape[0], device=device)
    mask = ((nn_sim >= threshold) & (ids1 == nn21[nn12]))
    matches = torch.stack([ids1[mask], nn12[mask]])
    return matches.t().data.cpu().numpy()


def similarity_matcher(descriptors1, descriptors2, threshold=0.9):
    # Similarity threshold matcher for L2 normalized descriptors.
    device = descriptors1.device
    
    sim = descriptors1 @ descriptors2.t()
    nn_sim, nn12 = torch.max(sim, dim=1)
    nn_dist = torch.sqrt(2 - 2 * nn_sim)
    nn21 = torch.max(sim, dim=0)[1]
    ids1 = torch.arange(0, sim.shape[0], device=device)
    mask = (nn_sim >= threshold)
    matches = torch.stack([ids1[mask], nn12[mask]])
    return matches.t(), nn_dist[mask]

def ratio_mutual_nn_matcher(descriptors1, descriptors2, ratio=0.92):
    # Lowe's ratio matcher + mutual NN for L2 normalized descriptors.
    device = descriptors1.device
    sim = descriptors1 @ descriptors2.t()
    nns_sim, nns = torch.topk(sim, 2, dim=1)
    nn12 = nns[:, 0]
    nns_dist = torch.sqrt(2 - 2 * nns_sim)
    nn21 = torch.max(sim, dim=0)[1]
    ids1 = torch.arange(0, sim.shape[0], device=device)
    matches = torch.stack([ids1, nns[:, 0]])
    ratios = nns_dist[:, 0] / (nns_dist[:, 1] + 1e-8)
    mask = torch.min(ids1 == nn21[nn12], ratios <= ratio)
    matches = matches[:, mask]
    return matches.t().data.cpu().numpy(), nns_dist[mask, 0]

def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    print("\n>> Creating net = " + checkpoint['net']) 
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)
    print(f" ( Model size: {nb_of_weights/1000:.0f}K parameters )")

    # initialization
    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()


class NonMaxSuppression (torch.nn.Module):
    def __init__(self, rel_thr=0.7, rep_thr=0.7):
        nn.Module.__init__(self)
        self.max_filter = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.rel_thr = rel_thr
        self.rep_thr = rep_thr
    
    def forward(self, reliability, repeatability, **kw):
        assert len(reliability) == len(repeatability) == 1
        reliability, repeatability = reliability[0], repeatability[0]

        # local maxima
        maxima = (repeatability == self.max_filter(repeatability))

        # remove low peaks
        maxima *= (repeatability >= self.rep_thr)
        maxima *= (reliability   >= self.rel_thr)

        return maxima.nonzero().t()[2:4]


def extract_multiscale( net, img, detector, scale_f=2**0.25, 
                        min_scale=0.0, max_scale=1, 
                        min_size=256, max_size=1280, 
                        verbose=False):
    old_bm = torch.backends.cudnn.benchmark 
    torch.backends.cudnn.benchmark = False # speedup
    
    # extract keypoints at multiple scales
    B, three, H, W = img.shape
    assert B == 1 and three == 3, "should be a batch with a single RGB image"
    
    assert max_scale <= 1
    s = 1.0 # current scale factor
    
    X,Y,S,C,Q,D = [],[],[],[],[],[]
    while  s+0.001 >= max(min_scale, min_size / max(H,W)):
        if s-0.001 <= min(max_scale, max_size / max(H,W)):
            nh, nw = img.shape[2:]
            if verbose: print(f"extracting at scale x{s:.02f} = {nw:4d}x{nh:3d}")
            # extract descriptors
            with torch.no_grad():
                res = net(imgs=[img])
                
            # get output and reliability map
            descriptors = res['descriptors'][0]
            reliability = res['reliability'][0]
            repeatability = res['repeatability'][0]

            # normalize the reliability for nms
            # extract maxima and descs
            y,x = detector(**res) # nms
            c = reliability[0,0,y,x]
            q = repeatability[0,0,y,x]
            d = descriptors[0,:,y,x].t()
            n = d.shape[0]

            # accumulate multiple scales
            X.append(x.float() * W/nw)
            Y.append(y.float() * H/nh)
            S.append((32/s) * torch.ones(n, dtype=torch.float32, device=d.device))
            C.append(c)
            Q.append(q)
            D.append(d)
        s /= scale_f

        # down-scale the image for next iteration
        break
        nh, nw = round(H*s), round(W*s)
        img = F.interpolate(img, (nh,nw), mode='bilinear', align_corners=False)

    # restore value
    torch.backends.cudnn.benchmark = old_bm

    Y = torch.cat(Y)
    X = torch.cat(X)
    S = torch.cat(S) # scale
    scores = torch.cat(C) * torch.cat(Q) # scores = reliability * repeatability
    XYS = torch.stack([X,Y,S], dim=-1)
    D = torch.cat(D)
    return XYS, D, scores


def extract_keypoints(img, args):
    t1 = time.time()

        
    xys, desc, scores = extract_multiscale(net, img, detector,
        scale_f   = args['scale_f'], 
        min_scale = args['min_scale'], 
        max_scale = args['max_scale'],
        min_size  = args['min_size'], 
        max_size  = args['max_size'], 
        verbose = False)

    xys = xys.cpu().numpy()
    scores = scores.cpu().numpy()
    idxs = np.argwhere(scores>0.85)

    return (xys[idxs], desc[idxs])
    

args = {'model' : 'r2d2/models/r2d2_WAF_N16.pt', 'scale_f' : 2**0.25, 'min_size' : 256, 'max_size' : 1380, 'min_scale' : 0, 'max_scale' : 1, 'reliability_thr' : 0.7, 'repeatability_thr' : 0.7 , 'gpu' : [0]}
net = load_network(args['model'])
iscuda = common.torch_set_gpu(args['gpu'])
if iscuda: net = net.cuda()
detector = NonMaxSuppression( rel_thr = args['reliability_thr'], rep_thr = args['repeatability_thr'])


def extract_features_and_desc(image):
    '''
    image: np.uint8
    '''
    # img_pil = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(image)
    img_cpu = img_pil
    # print("type(img_cpu):",type(img_cpu))
    img = norm_RGB(img_cpu)[None]
    if iscuda: 
      img = img.cuda()
    kps, desc = extract_keypoints(img, args)

    return np.squeeze(kps), np.squeeze(desc)

def get_matches(ref_kp, ref_desc, cur_kp, cur_desc, imgshape):
    matches = ratio_mutual_nn_matcher(ref_desc, cur_desc)[0]
    return matches




>> Creating net = Quad_L2Net_ConfCFS()
 ( Model size: 486K parameters )


/home/olorin/env/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


KeyError: 'HOSTNAME'

In [4]:
import numpy as np
import cv2

def umeyama_alignment(x, y, with_scale=True):
    """
    Computes the least squares solution parameters of an Sim(m) matrix
    that minimizes the distance between a set of registered points.
    Umeyama, Shinji: Least-squares estimation of transformation parameters
                     between two point patterns. IEEE PAMI, 1991
    :param x: mxn matrix of points, m = dimension, n = nr. of data points
    :param y: mxn matrix of points, m = dimension, n = nr. of data points
    :param with_scale: set to True to align also the scale (default: 1.0 scale)
    :return: r, t, c - rotation matrix, translation vector and scale factor
    """
    if x.shape != y.shape:
        assert False, "x.shape not equal to y.shape"

    # m = dimension, n = nr. of data points
    m, n = x.shape

    # means, eq. 34 and 35
    mean_x = x.mean(axis=1)
    mean_y = y.mean(axis=1)

    # variance, eq. 36
    # "transpose" for column subtraction
    sigma_x = 1.0 / n * (np.linalg.norm(x - mean_x[:, np.newaxis])**2)

    # covariance matrix, eq. 38
    outer_sum = np.zeros((m, m))
    for i in range(n):
        outer_sum += np.outer((y[:, i] - mean_y), (x[:, i] - mean_x))
    cov_xy = np.multiply(1.0 / n, outer_sum)

    # SVD (text betw. eq. 38 and 39)
    u, d, v = np.linalg.svd(cov_xy)

    # S matrix, eq. 43
    s = np.eye(m)
    if np.linalg.det(u) * np.linalg.det(v) < 0.0:
        # Ensure a RHS coordinate system (Kabsch algorithm).
        s[m - 1, m - 1] = -1

    # rotation, eq. 40
    r = u.dot(s).dot(v)

    # scale & translation, eq. 42 and 41
    c = 1 / sigma_x * np.trace(np.diag(d).dot(s)) if with_scale else 1.0
    t = mean_y - np.multiply(c, r.dot(mean_x))

    return r, t, c


def normalize_kp(kp, cam_intr):
    kp_norm = kp.copy()
    kp_norm[:, 0] = \
        (kp[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp_norm[:, 1] = \
        (kp[:, 1] - cam_intr[1,2]) / cam_intr[1,1]

    kp1_3D = np.ones((3, kp_norm.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp_norm[:, 0].copy(), kp_norm[:, 1].copy()

    return kp1_3D

def triangulation(kp1, kp2, T_1w, T_2w, cam_intr):
    """Triangulation to get 3D points
    Args:
        kp1 (Nx2): keypoint in view 1 (normalized)
        kp2 (Nx2): keypoints in view 2 (normalized)
        T_1w (4x4): pose of view 1 w.r.t  i.e. T_1w (from w to 1)
        T_2w (4x4): pose of view 2 w.r.t world, i.e. T_2w (from w to 2)
    Returns:
        X (3xN): 3D coordinates of the keypoints w.r.t world coordinate
        X1 (3xN): 3D coordinates of the keypoints w.r.t view1 coordinate
        X2 (3xN): 3D coordinates of the keypoints w.r.t view2 coordinate
    """
    
    kp1_norm = kp1.copy()
    kp2_norm = kp2.copy()

    kp1_norm[:, 0] = \
        (kp1[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp1_norm[:, 1] = \
        (kp1[:, 1] - cam_intr[1,2]) / cam_intr[1,1]
    kp2_norm[:, 0] = \
        (kp2[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp2_norm[:, 1] = \
        (kp2[:, 1] - cam_intr[1,2]) / cam_intr[1,1]

    kp1_3D = np.ones((3, kp1_norm.shape[0]))
    kp2_3D = np.ones((3, kp2_norm.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp1_norm[:, 0].copy(), kp1_norm[:, 1].copy()
    kp2_3D[0], kp2_3D[1] = kp2_norm[:, 0].copy(), kp2_norm[:, 1].copy()

    X = cv2.triangulatePoints(T_1w[:3], T_2w[:3], kp1_3D[:2], kp2_3D[:2])
    X /= X[3]
    X1 = T_1w[:3] @ X
    X2 = T_2w[:3] @ X
    return X[:3].T, X1, X2

# Taken from  DF-VO : https://github.com/Huangying-Zhan/DF-VO

import numpy as np

class SE3():
    """SE3 object consists rotation and translation components
    Attributes:
        pose (4x4 numpy array): camera pose
        inv_pose (4x4 numpy array): inverse camera pose
        R (3x3 numpy array): Rotation component
        t (3x1 numpy array): translation component,
    """
    def __init__(self, np_arr=None):
        if np_arr is None:
            self._pose = np.eye(4)
        else:
            self._pose = np_arr

    @property
    def pose(self):
        """ pose (4x4 numpy array): camera pose """
        return self._pose

    @pose.setter
    def pose(self, value):
        self._pose = value

    @property
    def inv_pose(self):
        """ inv_pose (4x4 numpy array): inverse camera pose """
        return np.linalg.inv(self._pose)

    @inv_pose.setter
    def inv_pose(self, value):
        self._pose = np.linalg.inv(value)

    @property
    def R(self):
        return self._pose[:3, :3]

    @R.setter
    def R(self, value):
        self._pose[:3, :3] = value

    @property
    def t(self):
        return self._pose[:3, 3:]

    @t.setter
    def t(self, value):
        self._pose[:3, 3:] = value

import numpy as np
# from sim3 import SE3

class FramePair:
    def __init__(self, f1, f2, matches_no, left_kp, right_kp, cheirality_pts_ct=0, inlier_pts_ct=0, pose=SE3()):
        self.frame1 = f1
        self.frame2 = f2
        # self.frame1.kps = left_kp
        # self.frame2.kps = right_kp
        self.left_kp = left_kp
        self.right_kp = right_kp
        self.cheirality_pts_ct = cheirality_pts_ct
        self.inlier_pts_ct = inlier_pts_ct
        self.pose = pose
        self.avg_optical_flow = 0
        self.matches_no = matches_no
        self.ess_mat = None


    def getpose(self):
        # return self.pose
        pos = SE3()
        pos.t = self.pose.t.copy()
        pos.R = self.pose.R.copy()
        return pos

    def getkeypts(self):
        return (self.left_kp, self.right_kp)
    
    def getchecks(self):
        return (self.cheirality_pts_ct, self.inlier_pts_ct)

class Frame:
    def __init__(self, id, img, kps, desc, fil,  pose = None, depth = np.zeros(1), image_arr = None, ):
        self.id = id
        self.image = img
        self.keypoints = kps
        self.descriptors = desc
        self.filename = fil
        self.depth = depth
        self.pose = pose
        self.image_arr = image_arr
        self.glob_pose = None
        self.tracked_kps = None
        self.global_pose = None
        self.inlier_pts_r = None
        self.inlier_pts_rl = None
    
    def getitems(self):
        return (self.image, self.keypoints, self.descriptors, self.filename)

    def get_kp_desc(self):
        return (self.keypoints, self.descriptors)

    def get_image(self):
        return self.image

    def get_file(self):
        return self.filename

In [ ]:

# %%writefile /content/Vis_Odometry/main_cell.py
# Parking Spot
import cv2
import numpy as np
import time
import copy
import pickle
from tqdm import tqdm
import sys
import glob
import re
import shutil
from sklearn import linear_model
import os
# import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, precision = 3)
from scipy.spatial.transform import Rotation as sciPyR
# from SURF import *
# from ORB import *

# from geom_utils import *
# from array_utils import *
# from frame_utils import *

listlength = 1

def append_to_list(l, ele, listlen= listlength):
    l.append(ele)
    return l[-listlen:]



class VisualOdometry:
        def __init__(self, camera_intrinsics, seq):

 
                self.cam_intr = camera_intrinsics.copy()            

                self.ref_data = []
                self.global_poses = {0: SE3().pose}
                self.global_pose = SE3()
                self.pose_ctr = 0
                self.cur_data = None
                self.frame_pairs = []
                self.viz = True
                self.scale = 1
                self.img_id = 0

                self.bad_pnp = 0

                self.no_matches = []
                self.inlier_pts = []
                self.cheirality_pts = []

                self.ref_kps_index = None
                self.ref_pose = SE3()
                self.good_points_3D = None

                self.initialized = False
                self.pose_ctr = 0
                self.ref_len = 1
                self.method = 'r2d2'

                self.seq = seq 

                self.essentialMat = {'iter' : 3, 'thresh' : 0.3, 'prob' : 0.99}

                if os.path.exists(self.method.lower()) == False:
                        os.mkdir(self.method.lower())


        def update_global_pose(self, poss):
            self.global_pose.t += self.global_pose.R @ poss.t
            self.global_pose.R = self.global_pose.R @ poss.R


        def computepose_3D_2D(self,framepair):

                best_inlier_cnt = 0
                retval = False
                pose = SE3()

                left_kp, right_kp = framepair.getkeypts()

                E1, inliers = cv2.findEssentialMat(right_kp.copy(), left_kp.copy(), self.cam_intr, method = cv2.RANSAC, prob = self.essentialMat['prob'], threshold = 4.0)

                inliers = np.squeeze(inliers)
                inliers[:] = 1
                inliers_copy_og = inliers.copy()

                left_kp = left_kp.copy()[inliers_copy_og==1]
                right_kp = right_kp.copy()[inliers_copy_og==1]

                left_depth_image = framepair.frame1.depth.copy()
                threeDImage = cv2.rgbd.depthTo3d(left_depth_image.astype(np.float32), self.cam_intr)
                left_3D_points = threeDImage[left_kp[...,1].astype(np.int32), left_kp[...,0].astype(np.int32)]

                good_3D_mask = (left_3D_points[..., 2] > 0) & (left_3D_points[..., 2] < 50)

                left_3D_points = left_3D_points[good_3D_mask]
                left_kp = left_kp[good_3D_mask]
                right_kp = right_kp[good_3D_mask]

                framepair.left_kp = left_kp.copy()
                framepair.right_kp = right_kp.copy()
                framepair.left_kp_og = left_kp.copy()
                framepair.right_kp_og = right_kp.copy()

                self.vizualize_custom_matches(framepair.frame1.image, framepair.frame2.image, left_kp, right_kp)

                # print(left_3D_points[::10], right_kp[::10])
                # self.vizualize_kps(None, self.cur_data.image, right_kp[::10], '_%06d'%(self.img_id))
                # done
                # print(framepair.frame1.filename, framepair.frame2.filename, left_3D_points)


                best_rt = []
                best_inliers = None
                best_inlier = 0
                
                for loopindex in range(self.essentialMat['iter']):
                     
                        new_list = np.random.randint(0, left_3D_points.shape[0], (left_3D_points.shape[0]))
                        new_left_kp = left_kp.copy()[new_list]
                        new_right_kp = right_kp.copy()[new_list]
                        new_points_3D = left_3D_points.copy()[new_list]

                        flag, r, t, inlier = cv2.solvePnPRansac(objectPoints = new_points_3D, imagePoints = new_right_kp, cameraMatrix = self.cam_intr, distCoeffs = None, iterationsCount = 100, reprojectionError=1.500)
                        # print(flag)
                        if flag and inlier.shape[0] > best_inlier and inlier.shape[0] > 30:
                                best_rt = [r, t]
                                best_inlier = inlier.shape[0]
                                best_inliers = np.squeeze(inlier).copy()

                pose = SE3()
                if len(best_rt) != 0:
                        retval = True
                        r, t = best_rt
                        pose.R = cv2.Rodrigues(r)[0]
                        pose.t = t
                        pose.pose = pose.inv_pose.copy()
                        framepair.pose = pose
                else:
                    bad=True
                    print("NO IT IS A BAD PNP")

                return retval, framepair, len(left_kp), best_inlier



        def vizualize_matches(self,framepair, imgidx, val):
                cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in framepair.left_kp]
                cv_kp2 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in framepair.right_kp]
                dmtches = [cv2.DMatch(_imgIdx=0,_queryIdx=i, _trainIdx=i, _distance = 0) for i in range(len(cv_kp1))]
                out_img = cv2.drawMatches(framepair.frame1.image, cv_kp1, framepair.frame2.image, cv_kp2, dmtches[::50], None, (0,255,255), -1, None, 2)
                cv2.imwrite('match', out_img)


        def vizualize_custom_matches(self,img1, img2, kps1, kps2, name = 'match_inl'):
                cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in kps1]
                cv_kp2 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in kps2]
                dmtches = [cv2.DMatch(_imgIdx=0,_queryIdx=i, _trainIdx=i, _distance = 0) for i in range(len(cv_kp1))]
                out_img = cv2.drawMatches(img1, cv_kp1, img2, cv_kp2, dmtches[::10], None, (0,255,255), -1, None, 2)
                cv2.imwrite('outfolder/%02d_%06d_match_inl.jpg'%(self.seq, self.img_id), out_img)


        def vizualize_kps(self,img1, img2, kps, val, to_update = False):

              cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=5) for pt in kps]
              if img1!=None:
                out_img = cv2.drawKeypoints(img1.astype(np.uint8), cv_kp1, None, (0,255,255))
                cv2_imshow(out_img)
                cv2.imwrite('outfolder/%06d_d.jpg'%val, out_img)
              out_img = cv2.drawKeypoints(img2.astype(np.uint8), cv_kp1, None, (0,255,255))
              cv2.imwrite('outfolder/' + '%s_c.jpg'%val, out_img)

        def update_frames_data(self, framepair):
                self.ref_data = append_to_list(self.ref_data, self.cur_data, 2)
        
        def update_framepairs(self, fp):
                self.frame_pairs = append_to_list(self.frame_pairs, fp, 4)


        def get_point_in_other_image(self, framepair, to_update = False):

            temp_pose = framepair.frame2.pose.inv_pose
            projectedpts = cv2.projectPoints(self.midpoint_3D, cv2.Rodrigues(temp_pose[:3, :3])[0], temp_pose[:3,3], self.cam_intr, None)[0]
            self.vizualize_kps(None, self.cur_data.image, projectedpts[0].astype(np.int32), '_%06d'%(self.img_id), to_update)


        def save_poses(self, save_name = 'r2d2parking.pkl'):
                with open(save_name, 'wb') as f: pickle.dump(self.global_poses, f)


        def process_frame(self, img, depth_img, frame_no):
                '''
                contours: Area within which keypoints are to be tracked
                img: OpenCV image
                midpoint: Midpoint to be tracked
                frame_no: Frame number
                '''
                #NOTE:
                #Use midpoint and contours for 2 frames i.e until we have established 2D-2D correspondencies
                #Followed by this use pure Visual Odometry for servoing

                #For the first frame
                if(frame_no == 0): #First frame
                        if self.method != 'OF':
                                kp, desc = extract_features_and_desc(img)                
                                frame1 = Frame(id = 0, img = img, kps = kp, desc = desc, fil = '%06d'%frame_no, pose = SE3(), depth = depth_img)

                        self.ref_data = append_to_list(self.ref_data, frame1)
                        self.initialized = False
                        return SE3()
                
                else: #NOTE: frame_no = imgidx
                        cur_img = img
                        imgidx = frame_no
                        cur_fil = ''
                        self.img_id = imgidx
                        to_update = False
                                
                        cur_kp, cur_desc = extract_features_and_desc(cur_img)
                        frame1 = self.ref_data[-1]
                        ref_kp, ref_desc = self.ref_data[-1].get_kp_desc()
                        frame2 = Frame(id = imgidx, img = cur_img, kps = cur_kp, desc = cur_desc, fil = '%06d'%frame_no, pose = SE3(), depth = depth_img)
                        self.cur_data = frame2

                        matches = get_matches(ref_kp, ref_desc, cur_kp, cur_desc, cur_img.shape)
                        ref_keypoints = ref_kp[matches[:, 0], : 2].astype(np.float32)
                        cur_keypoints = cur_kp[matches[:, 1], : 2].astype(np.float32)

                        diff = np.linalg.norm(ref_keypoints- cur_keypoints, axis=1)
                        no = len(diff[diff>=10])
                        difference_perc = no*100/len(diff)
                        if difference_perc < 30:
                          compute_flag = False
                        cur_keypoints = cur_keypoints[diff>=5]
                        ref_keypoints = ref_keypoints[diff>=5]


                        framepair = FramePair(frame1, frame2, matches, ref_keypoints, cur_keypoints, matches)
                        
                        try:
                            t1 = time.time()
                            retval, framepair, common_pts, best_inliers = self.computepose_3D_2D(framepair)
                            dist_scale = np.linalg.norm(framepair.pose.t)
                        except Exception as e:
                            print(e)
                            # aosan
                            self.bad_pnp += 1
                            retval = False

                        if retval:   
                            self.bad_pnp = 0
                            framepair.frame2.pose._pose = framepair.frame1.pose._pose @ framepair.pose._pose.copy()
                            if (common_pts < 200 or best_inliers < 100 or dist_scale > 1.5) and difference_perc > 50:
                              to_update = True

                        else:
                          framepair.frame2.pose._pose = framepair.frame1.pose._pose.copy()

                        self.pose_ctr += 1
                        self.global_poses[self.pose_ctr] = framepair.frame2.pose._pose
                        # print("CUR POSE ", framepair.frame2.pose.t.T)

                        if to_update or self.bad_pnp >3:
                              # print("UPDATING", frame_no)
                              self.update_frames_data(framepair)

                        return framepair.frame2.pose





In [ ]:
midpoints = [(168, 353), (175, 328), (183, 313), (204, 306), (187, 296), (203, 292), (229, 279), (235, 269), (235, 269)]
numpyseed = 8214

for seq_no in range(0, 9):
    np.random.seed(numpyseed)
    img_dir = "all_sequences/%02d/"%(seq_no)
    strartno = 1
    left_image_files = sorted(glob.glob(img_dir + '/left*'))[strartno::4]
    dep_image_files = sorted(glob.glob(img_dir + '/depth*'))[strartno::4]
    cam_intr = np.asarray([[332.9648157406122, 0.0, 310.8472797033171], [0.0, 444.0950902369522, 252.76060777256825], [0.0, 0.0, 1.0]])
    vo = VisualOdometry(cam_intr, seq_no)
    for index, (left_image_file, dep_image_file) in enumerate(zip(left_image_files, dep_image_files)):
        left_frame = cv2.imread(left_image_file)
        left_frame = cv2.cvtColor(left_frame, cv2.COLOR_BGR2RGB)
        left_frame = cv2.resize(left_frame, (640,480), interpolation = cv2.INTER_LANCZOS4)
        dep_img = np.load(dep_image_file)
        dep_img = cv2.resize(dep_img, (640,480), interpolation = cv2.INTER_NEAREST)
        frame_pose = vo.process_frame(left_frame, dep_img, index)
    print(frame_pose.t.T)